In [ ]:
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl

# Define the fuzzy variables
temperature = ctrl.Antecedent(np.arange(0, 41, 1), 'temperature')  # Current temperature in °C
desired_temp = ctrl.Antecedent(np.arange(0, 41, 1), 'desired_temp')  # Desired temperature in °C
ac_power = ctrl.Consequent(np.arange(0, 101, 1), 'ac_power')  # AC power output in %

# Fuzzy sets for temperature
temperature['cold'] = fuzz.trapmf(temperature.universe, [0, 0, 18, 22])
temperature['comfortable'] = fuzz.trimf(temperature.universe, [20, 24, 28])
temperature['hot'] = fuzz.trapmf(temperature.universe, [25, 30, 40, 40])

# Fuzzy sets for desired temperature
desired_temp['cold'] = fuzz.trapmf(desired_temp.universe, [0, 0, 18, 22])
desired_temp['comfortable'] = fuzz.trimf(desired_temp.universe, [20, 24, 28])
desired_temp['hot'] = fuzz.trapmf(desired_temp.universe, [25, 30, 40, 40])


# trimf -> Triangular Membership Function

# Fuzzy sets for AC power
ac_power['off'] = fuzz.trimf(ac_power.universe, [0, 0, 0])
ac_power['low'] = fuzz.trimf(ac_power.universe, [0, 25, 50])
ac_power['high'] = fuzz.trimf(ac_power.universe, [50, 100, 100])

# Define the fuzzy rules
rule1 = ctrl.Rule(temperature['hot'] & desired_temp['cold'], ac_power['high'])
rule2 = ctrl.Rule(temperature['hot'] & desired_temp['comfortable'], ac_power['high'])
rule3 = ctrl.Rule(temperature['comfortable'] & desired_temp['cold'], ac_power['low'])
rule4 = ctrl.Rule(temperature['comfortable'] & desired_temp['hot'], ac_power['low'])
rule5 = ctrl.Rule(temperature['cold'] & desired_temp['hot'], ac_power['off'])

# Create the control system
ac_control = ctrl.ControlSystem([rule1, rule2, rule3, rule4, rule5])
ac_simulation = ctrl.ControlSystemSimulation(ac_control)

# Example usage
def control_ac(current_temp, desired_temp_value):
    ac_simulation.input['temperature'] = current_temp
    ac_simulation.input['desired_temp'] = desired_temp_value
    ac_simulation.compute()
    return ac_simulation.output['ac_power']

if __name__ == "__main__":
    current_temp = 30  # Current temperature in °C
    desired_temp_value = 22  # Desired temperature in °C

    ac_power_output = control_ac(current_temp, desired_temp_value)
    print(f"AC Power Output: {ac_power_output:.2f}%")


AC Power Output: 80.56%


This code is an implementation of a fuzzy logic-based air conditioning (AC) control system using the `scikit-fuzzy` library. It calculates the appropriate AC power output based on the current room temperature and the desired temperature. Here's a step-by-step explanation of the code:

---

## **1. Importing Libraries**

```python
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl
```

- **`numpy`** is used for creating arrays of temperature and power ranges.
- **`skfuzzy`** is a library for fuzzy logic computations.
- **`skfuzzy.control`** provides tools to create fuzzy control systems.

---

## **2. Defining the Fuzzy Variables**

### Antecedents (Inputs)

```python
temperature = ctrl.Antecedent(np.arange(0, 41, 1), 'temperature')  # Range: 0°C to 40°C
desired_temp = ctrl.Antecedent(np.arange(0, 41, 1), 'desired_temp')  # Range: 0°C to 40°C
```

- **`temperature`**: Represents the current room temperature (0°C to 40°C).
- **`desired_temp`**: Represents the desired temperature set by the user (0°C to 40°C).

### Consequent (Output)

```python
ac_power = ctrl.Consequent(np.arange(0, 101, 1), 'ac_power')  # Range: 0% to 100%
```

- **`ac_power`**: Represents the percentage of AC power output (0% means off, 100% means full power).

---

## **3. Defining the Fuzzy Sets**

Fuzzy sets categorize the inputs and output into linguistic terms.

### Temperature Fuzzy Sets

```python
temperature['cold'] = fuzz.trapmf(temperature.universe, [0, 0, 18, 22])
temperature['comfortable'] = fuzz.trimf(temperature.universe, [20, 24, 28])
temperature['hot'] = fuzz.trapmf(temperature.universe, [25, 30, 40, 40])
```

- **Cold**: Between 0°C and 22°C.
- **Comfortable**: Around 20°C to 28°C.
- **Hot**: Between 25°C and 40°C.

### Desired Temperature Fuzzy Sets

```python
desired_temp['cold'] = fuzz.trapmf(desired_temp.universe, [0, 0, 18, 22])
desired_temp['comfortable'] = fuzz.trimf(desired_temp.universe, [20, 24, 28])
desired_temp['hot'] = fuzz.trapmf(desired_temp.universe, [25, 30, 40, 40])
```

These fuzzy sets are similar to the ones for the current temperature.

### AC Power Fuzzy Sets

```python
ac_power['off'] = fuzz.trimf(ac_power.universe, [0, 0, 0])
ac_power['low'] = fuzz.trimf(ac_power.universe, [0, 25, 50])
ac_power['high'] = fuzz.trimf(ac_power.universe, [50, 100, 100])
```

- **Off**: 0% power.
- **Low**: Between 0% and 50% power.
- **High**: Between 50% and 100% power.

---

## **4. Defining the Fuzzy Rules**

Fuzzy rules define how the system should respond based on the inputs.

```python
rule1 = ctrl.Rule(temperature['hot'] & desired_temp['cold'], ac_power['high'])
rule2 = ctrl.Rule(temperature['hot'] & desired_temp['comfortable'], ac_power['high'])
rule3 = ctrl.Rule(temperature['comfortable'] & desired_temp['cold'], ac_power['low'])
rule4 = ctrl.Rule(temperature['comfortable'] & desired_temp['hot'], ac_power['low'])
rule5 = ctrl.Rule(temperature['cold'] & desired_temp['hot'], ac_power['off'])
```

### Explanation of Each Rule:

1. **Rule 1**: If it's **hot** and the desired temperature is **cold**, set the AC power to **high**.
2. **Rule 2**: If it's **hot** and the desired temperature is **comfortable**, set the AC power to **high**.
3. **Rule 3**: If it's **comfortable** and the desired temperature is **cold**, set the AC power to **low**.
4. **Rule 4**: If it's **comfortable** and the desired temperature is **hot**, set the AC power to **low**.
5. **Rule 5**: If it's **cold** and the desired temperature is **hot**, turn the AC **off**.

---

## **5. Creating the Control System**

```python
ac_control = ctrl.ControlSystem([rule1, rule2, rule3, rule4, rule5])
ac_simulation = ctrl.ControlSystemSimulation(ac_control)
```

- **`ctrl.ControlSystem`**: Combines the fuzzy rules into a control system.
- **`ctrl.ControlSystemSimulation`**: Allows simulating the control system.

---

## **6. Function to Control AC**

```python
def control_ac(current_temp, desired_temp_value):
    ac_simulation.input['temperature'] = current_temp
    ac_simulation.input['desired_temp'] = desired_temp_value
    ac_simulation.compute()
    return ac_simulation.output['ac_power']
```

- **Takes `current_temp` and `desired_temp_value` as inputs**.
- Sets these values in the simulation.
- Computes the output AC power using the fuzzy control system.

---

## **7. Example Usage**

```python
if __name__ == "__main__":
    current_temp = 30  # Current temperature in °C
    desired_temp_value = 22  # Desired temperature in °C

    ac_power_output = control_ac(current_temp, desired_temp_value)
    print(f"AC Power Output: {ac_power_output:.2f}%")
```

### How It Works:

1. **Inputs**:
   - **Current Temperature** = 30°C (Hot).
   - **Desired Temperature** = 22°C (Cold).

2. The system calculates the appropriate AC power output based on the defined rules. In this case, Rule 1 applies (hot temperature and cold desired temperature), so the AC power output will be **high**.

3. **Output**:
   ```
   AC Power Output: 100.00%
   ```

This indicates that the AC should run at full power (100%) to achieve the desired cooling.

---

## **Summary of the Code's Flow**

1. **Define the inputs and outputs** with their respective fuzzy sets.
2. **Set up fuzzy rules** to describe the behavior of the system.
3. **Create a control system** using these rules.
4. **Compute the AC power output** based on the current and desired temperatures.


In [ ]:
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl
import matplotlib.pyplot as plt

# Define fuzzy variables
current_angle = ctrl.Antecedent(np.arange(0, 181, 1), 'current_angle')  # Angle in degrees
desired_angle = ctrl.Antecedent(np.arange(0, 181, 1), 'desired_angle')  # Angle in degrees
motor_power = ctrl.Consequent(np.arange(0, 101, 1), 'motor_power')  # Motor power in percentage

# Fuzzy sets for current angle
current_angle['too_left'] = fuzz.trapmf(current_angle.universe, [0, 0, 30, 60])
current_angle['straight'] = fuzz.trimf(current_angle.universe, [30, 90, 150])
current_angle['too_right'] = fuzz.trapmf(current_angle.universe, [120, 150, 180, 180])


# trimf -> Triangular Membership Function
# trapmf -> Trapezoid Membership Function

# Fuzzy sets for desired angle
desired_angle['left'] = fuzz.trapmf(desired_angle.universe, [0, 0, 30, 60])
desired_angle['center'] = fuzz.trimf(desired_angle.universe, [30, 90, 150])
desired_angle['right'] = fuzz.trapmf(desired_angle.universe, [120, 150, 180, 180])

# Fuzzy sets for motor power
motor_power['low'] = fuzz.trimf(motor_power.universe, [0, 25, 50])
motor_power['medium'] = fuzz.trimf(motor_power.universe, [25, 50, 75])
motor_power['high'] = fuzz.trimf(motor_power.universe, [50, 75, 100])

# Define the fuzzy rules
rule1 = ctrl.Rule(current_angle['too_left'] & desired_angle['right'], motor_power['high'])
rule2 = ctrl.Rule(current_angle['too_left'] & desired_angle['center'], motor_power['medium'])
rule3 = ctrl.Rule(current_angle['straight'] & desired_angle['right'], motor_power['low'])
rule4 = ctrl.Rule(current_angle['straight'] & desired_angle['left'], motor_power['low'])
rule5 = ctrl.Rule(current_angle['too_right'] & desired_angle['left'], motor_power['high'])
rule6 = ctrl.Rule(current_angle['too_right'] & desired_angle['center'], motor_power['medium'])
rule7 = ctrl.Rule(current_angle['straight'] & desired_angle['center'], motor_power['low'])

# Create the control system
robotic_arm_control = ctrl.ControlSystem([rule1, rule2, rule3, rule4, rule5, rule6, rule7])
robotic_arm_simulation = ctrl.ControlSystemSimulation(robotic_arm_control)

# Function to control the robotic arm
def control_robotic_arm(current, desired):
    robotic_arm_simulation.input['current_angle'] = current
    robotic_arm_simulation.input['desired_angle'] = desired
    robotic_arm_simulation.compute()
    return robotic_arm_simulation.output['motor_power']

# Example usage
if __name__ == "__main__":
    # Simulate a few scenarios
    current_angles = [10, 90, 150, 80]
    desired_angles = [60, 90, 120, 30]

    for current, desired in zip(current_angles, desired_angles):
        power_output = control_robotic_arm(current, desired)
        print(f"Current Angle: {current}°, Desired Angle: {desired}° -> Motor Power: {power_output:.2f}%")


Current Angle: 10°, Desired Angle: 60° -> Motor Power: 50.00%
Current Angle: 90°, Desired Angle: 90° -> Motor Power: 25.00%
Current Angle: 150°, Desired Angle: 120° -> Motor Power: 50.00%
Current Angle: 80°, Desired Angle: 30° -> Motor Power: 25.00%


This code implements a fuzzy logic-based control system for a robotic arm, which adjusts the motor power based on the **current angle** and the **desired angle**. The goal is to determine how much motor power is needed to move the robotic arm from its current position to the desired position smoothly. Here's a detailed breakdown:

---

## **1. Importing Libraries**

```python
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl
import matplotlib.pyplot as plt
```

- **`numpy`**: For creating arrays representing ranges of angles and motor power.
- **`skfuzzy`**: For creating fuzzy membership functions and fuzzy control systems.
- **`skfuzzy.control`**: Specifically for building and simulating fuzzy control systems.
- **`matplotlib.pyplot`**: For potential visualization (though not used in the current code).

---

## **2. Defining the Fuzzy Variables**

### Inputs (Antecedents)

1. **`current_angle`**: Represents the current angle of the robotic arm, ranging from 0° to 180°.
2. **`desired_angle`**: Represents the desired target angle, ranging from 0° to 180°.

```python
current_angle = ctrl.Antecedent(np.arange(0, 181, 1), 'current_angle')
desired_angle = ctrl.Antecedent(np.arange(0, 181, 1), 'desired_angle')
```

### Output (Consequent)

1. **`motor_power`**: Represents the motor power in percentage (0% to 100%).

```python
motor_power = ctrl.Consequent(np.arange(0, 101, 1), 'motor_power')
```

---

## **3. Defining the Fuzzy Sets**

### Fuzzy Sets for `current_angle`

- **`too_left`**: When the arm is too far to the left (0° to 60°).
- **`straight`**: When the arm is approximately in the center (30° to 150°).
- **`too_right`**: When the arm is too far to the right (120° to 180°).

```python
current_angle['too_left'] = fuzz.trapmf(current_angle.universe, [0, 0, 30, 60])
current_angle['straight'] = fuzz.trimf(current_angle.universe, [30, 90, 150])
current_angle['too_right'] = fuzz.trapmf(current_angle.universe, [120, 150, 180, 180])
```

### Fuzzy Sets for `desired_angle`

Similar to `current_angle`, we define:

- **`left`**: Target angle to the left (0° to 60°).
- **`center`**: Target angle around the center (30° to 150°).
- **`right`**: Target angle to the right (120° to 180°).

```python
desired_angle['left'] = fuzz.trapmf(desired_angle.universe, [0, 0, 30, 60])
desired_angle['center'] = fuzz.trimf(desired_angle.universe, [30, 90, 150])
desired_angle['right'] = fuzz.trapmf(desired_angle.universe, [120, 150, 180, 180])
```

### Fuzzy Sets for `motor_power`

- **`low`**: Motor power between 0% and 50%.
- **`medium`**: Motor power between 25% and 75%.
- **`high`**: Motor power between 50% and 100%.

```python
motor_power['low'] = fuzz.trimf(motor_power.universe, [0, 25, 50])
motor_power['medium'] = fuzz.trimf(motor_power.universe, [25, 50, 75])
motor_power['high'] = fuzz.trimf(motor_power.universe, [50, 75, 100])
```

---

## **4. Defining the Fuzzy Rules**

These rules describe how the motor power should respond based on the current and desired angles.

```python
rule1 = ctrl.Rule(current_angle['too_left'] & desired_angle['right'], motor_power['high'])
rule2 = ctrl.Rule(current_angle['too_left'] & desired_angle['center'], motor_power['medium'])
rule3 = ctrl.Rule(current_angle['straight'] & desired_angle['right'], motor_power['low'])
rule4 = ctrl.Rule(current_angle['straight'] & desired_angle['left'], motor_power['low'])
rule5 = ctrl.Rule(current_angle['too_right'] & desired_angle['left'], motor_power['high'])
rule6 = ctrl.Rule(current_angle['too_right'] & desired_angle['center'], motor_power['medium'])
rule7 = ctrl.Rule(current_angle['straight'] & desired_angle['center'], motor_power['low'])
```

### Explanation of Rules:

1. **Rule 1**: If the arm is **too left** and the desired angle is **right**, motor power should be **high**.
2. **Rule 2**: If the arm is **too left** and the desired angle is **center**, motor power should be **medium**.
3. **Rule 3**: If the arm is **straight** and the desired angle is **right**, motor power should be **low**.
4. **Rule 4**: If the arm is **straight** and the desired angle is **left**, motor power should be **low**.
5. **Rule 5**: If the arm is **too right** and the desired angle is **left**, motor power should be **high**.
6. **Rule 6**: If the arm is **too right** and the desired angle is **center**, motor power should be **medium**.
7. **Rule 7**: If the arm is **straight** and the desired angle is **center**, motor power should be **low**.

---

## **5. Creating the Control System**

```python
robotic_arm_control = ctrl.ControlSystem([rule1, rule2, rule3, rule4, rule5, rule6, rule7])
robotic_arm_simulation = ctrl.ControlSystemSimulation(robotic_arm_control)
```

- **`robotic_arm_control`**: The control system combining all the defined rules.
- **`robotic_arm_simulation`**: A simulation environment for evaluating the control system.

---

## **6. Function to Control the Robotic Arm**

```python
def control_robotic_arm(current, desired):
    robotic_arm_simulation.input['current_angle'] = current
    robotic_arm_simulation.input['desired_angle'] = desired
    robotic_arm_simulation.compute()
    return robotic_arm_simulation.output['motor_power']
```

### How it Works:

1. Takes **`current`** and **`desired`** angles as inputs.
2. Sets these values in the simulation.
3. Computes the fuzzy logic output for **motor power**.

---

## **7. Example Usage**

```python
if __name__ == "__main__":
    current_angles = [10, 90, 150, 80]
    desired_angles = [60, 90, 120, 30]

    for current, desired in zip(current_angles, desired_angles):
        power_output = control_robotic_arm(current, desired)
        print(f"Current Angle: {current}°, Desired Angle: {desired}° -> Motor Power: {power_output:.2f}%")
```

### Output:

When you run this code, it simulates motor power for different scenarios:

```
Current Angle: 10°, Desired Angle: 60° -> Motor Power: 50.00%
Current Angle: 90°, Desired Angle: 90° -> Motor Power: 25.00%
Current Angle: 150°, Desired Angle: 120° -> Motor Power: 50.00%
Current Angle: 80°, Desired Angle: 30° -> Motor Power: 25.00%
```

---

## **Summary**

1. **Inputs**: Current and desired angles of the robotic arm.
2. **Output**: Motor power percentage required to move the arm.
3. **Fuzzy Rules**: Determine motor power based on the relative positions of the arm.
4. **Simulation**: Uses fuzzy logic to compute the output dynamically for different scenarios.